In [1]:
import datetime as dt
import numpy as np
import matplotlib
matplotlib.use('Agg')
import matplotlib.pyplot as plt
import cartopy.feature
from cartopy.util import add_cyclic_point
import cartopy.crs as ccrs
import xarray as xr
import math
import netCDF4
#from get_ellipse_metrics import get_emetrics_max_min
from fitEllipse2_new import fitEllipseContour
from geopy.distance import great_circle
from matplotlib.patches import Polygon
from matplotlib import cm #colormaps!
import copy
import os
import scipy.stats
import pickle

#os.environ['DISPLAY']=':0.0'


In [2]:
#### Define functions first:
def point_inside_polygon(x,y,xarr,yarr):

	n = len(xarr)
	inside =False

	p1x = xarr[0]
	p1y = yarr[0]
	for i in range(n+1):
		p2x,p2y = xarr[i % n],yarr[i % n]
		if y > min(p1y,p2y):
			if y <= max(p1y,p2y):
				if x <= max(p1x,p2x):
					if p1y != p2y:
						xinters = (y-p1y)*(p2x-p1x)/(p2y-p1y)+p1x
					if p1x == p2x or x <= xinters:
						inside = not inside
		p1x,p1y = p2x,p2y

	return inside;


In [3]:
### The main code begins here....
home_dir = '/home11/grad/2020/ef935217/efernandez/Research/Ellipses/PORCorr'  # <----Change this 
lev_list = [10,50,30]
temp_lev = [850]
#contour_list = [31500,20000,23000]
contour_list = [30000,20000,23000]

write_out = True #write the ellipse data to a text file, True or False 

In [4]:
ephi_ratio10 = np.empty((1,366))
ephi_ratio10[:] = np.nan
ephi_wind10 = np.empty((1,366))
ephi_wind10[:] = np.nan
ephi_cenlat10 = np.empty((1,366))
ephi_cenlat10[:] = np.nan
ephi_cenlon10 = np.empty((1,366))
ephi_cenlon10[:] = np.nan
ephi_size10 = np.empty((1,366))
ephi_size10[:] = np.nan
ephi10 = np.empty((1,366))
ephi10[:] = np.nan


ephi_ratio50 = np.empty((1,366))
ephi_ratio50[:] = np.nan
ephi_wind50 = np.empty((1,366))
ephi_wind50[:] = np.nan
ephi_cenlat50 = np.empty((1,366))
ephi_cenlat50[:] = np.nan
ephi_cenlon50 = np.empty((1,366))
ephi_cenlon50[:] = np.nan
ephi_size50 = np.empty((1,366))
ephi_size50[:] = np.nan
ephi50 = np.empty((1,366))
ephi50[:] = np.nan

In [5]:
gfile = xr.open_dataset("/erai/climo/mean/g.mean.climo.nc")
gfile

<xarray.Dataset>
Dimensions:  (hour: 4, lat: 256, lev: 37, lon: 512, time: 366)
Coordinates:
  * time     (time) float64 1.0 2.0 3.0 4.0 5.0 ... 363.0 364.0 365.0 366.0
  * hour     (hour) float64 0.0 6.0 12.0 18.0
  * lev      (lev) int32 1000 975 950 925 900 875 850 825 ... 30 20 10 7 5 3 2 1
  * lat      (lat) float32 -89.46282 -88.76695 -88.06697 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.70312524 1.4062505 ... 358.59387 359.297
Data variables:
    g        (time, hour, lev, lat, lon) float32 ...

In [6]:
windlat = 60.0

In [7]:
for plot_lev,the_contour in zip(lev_list,contour_list): #loop through each level
	print("Opening ERAI data "+str(plot_lev)+" and controur level "+str(the_contour)  )
	
	rat = []
	cenl = []
	cenln = []
	wind = []
	sz = []
	ep = []
	
	plots_loc10 = "./ClimoPlots/10hPa/"   ##<-- Where are you saving the plot files?
	#plots_loc30 = "./IndivEventEllipse/Jan2019/30hPa/" 
	plots_loc50 = "./ClimoPlots/50hPa/" 
#	text_file = "./"+date1.strftime("%Y%m%d_%H%M")+"_"+str(plot_lev)+".txt"   ##<-- Where are you saving text files?
#	fout = open(text_file,'w')
    ##temp text
	#text_file2 = "./"+date1.strftime("%Y%m%d_%H%M")+"_temp"+str(plot_lev)+".txt"  
	#fout2 = open(text_file2,'w')
	if plot_lev == 10:
 
		gfile = xr.open_dataset("/erai/climo/mean/g.mean.climo.nc")
		g_files = gfile["g"]
		tfile = xr.open_dataset("/erai/climo/mean/t.mean.climo.nc")
		t_files = tfile["t"]
		ufile = xr.open_dataset("/erai/climo/mean/u.mean.climo.nc")
		u_files = ufile["u"]


		print("Partitioning and averaging data.")
		print("First, g data")
		g_data = g_files.loc[dict(lat=slice(0,90),lev=lev_list)]
		print("u data")
		u_data = u_files.loc[dict(lat=slice(windlat-1,windlat+1),lev=lev_list)].mean(dim='lon')
		print("t data")        
		t_data = t_files.loc[dict(lat=slice(75,90),lev=lev_list)]
		tlats = t_data['lat'].values
		weights=np.cos(np.deg2rad(t_data.lat))
		weights.name = "weights"

        
#hgt_250, lon = cutil.add_cyclic_point(data_hght.variables['Geopotential_height_isobaric'][:],coord=lon)
		print("lat and lon")
		lats_era = g_data["lat"].values
		lons_era = g_data["lon"].values

        
	print(f"level: {plot_lev}")

	for date in range(1,367): #loop through each day 
		print(f"date: {date}")
		t = date
		height = g_data.loc[dict(time=t,lev=plot_lev,hour=0.0)]
		u = u_data.loc[dict(time=t,lev=plot_lev)].mean(dim='lat')
		tdata = t_data.loc[dict(time=t,lev=plot_lev)].mean(dim='lon')   
		#tdata2 = t2_data.loc[dict(time=times[t])]
		temp = tdata.weighted(weights).mean(dim='lat')
        
		print("height",height)
		#print(tdata2)
		#stop
		#td2 = tdata2.values
		#for i in range(len(td2[:,1])):
			#fout2.write(str(td2[i,:])+'\n')            

              
		#formatted_date = times[date].strftime("%Y%m%d")
		emark = []
		eline = []
		cs_temp = []
		#mem_list = list[range(0,30),31]
		#mem_list.append(0)
		#mem_list.append(-1)
		print("##########################")
		print(date)
		#print("Date "+times[date].strftime("%H UTC %d %b %Y"))
		#valid_label = (times[date].strftime("Valid: %H UTC %d %b %Y"))
		
		
		plt.Figure(figsize=(15,15),dpi=120)#figure(figsize=(12,12),dpi=1200) <---Set fig size here!

		#ax = plt.subplot(1,1,1,projection=ccrs.Orthographic(0,90))
		ax = plt.axes(projection=ccrs.Orthographic(0,90))
		ax.add_feature(cartopy.feature.LAND, zorder=1, edgecolor='dimgray',facecolor='none')
		#ax.set_extent([-180,180,0,90],ccrs.PlateCarree())
		ax.outline_patch.set_edgecolor('none')
		gl = ax.gridlines(color="grey",linestyle=":",linewidth=0.5)#=lat_lon_width)

		clevs = range(18000,33500,250)

		[x,y] = np.meshgrid(lons_era,lats_era)
		#[x2,y2] = np.meshgrid(tlons_era,tlats_era)


		plt.title(f"{date}"+ " of 366 "+str(plot_lev)+"hPa Elliptical Diagnostics\n")

		mem_color = "blue"
		mem_lw = 3.0
		mem_ms = 8
		cont_color = "#aec2e2"
		cont_lw = 1.0
        
		cs_temp.append(ax.contour(x,y,height,levels=clevs,linewidths=cont_lw,colors=cont_color))
		

		line_w = np.array(clevs,dtype=np.float)
		for i,c in enumerate(clevs):
			if c == the_contour:
				line_w[i] = 1.5 # Make the contour we're using for the ellipse calc fatter
			else:
				line_w[i] = 0.5
		print(clevs)

		ax.contour(x,y,height,clevs,transform=ccrs.PlateCarree(),extend='both',colors='black',linewidths=line_w)
		#ax.contour(x2,y2,tdata2, lev = [tmean], transform=ccrs.PlateCarree(),extend='both',colors='blue',linewidths=line_w)
##NEED TO INCORPORATE THE CLIMATOLOGICAL MEAN TEMP LINE AT 850 MB
		try:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			print("in try")
			print("Number of ellipses: ",len(isoline_list))                                
		except:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []        
        
		if len(isoline_list) > 0:
			lev_contour_ind = np.where(np.array(cs_temp[-1].levels)==the_contour)[0][0]
			isoline_list = cs_temp[-1].allsegs[lev_contour_ind]
			#print(isoline_list)
			print("in elipse ...")
			#quit()
			print("Number of ellipses: ",len(isoline_list))                                
		else:
			print("Contours not found for",the_contour,"meter level at",plot_lev,"hPa.")
			isoline_list = []

		isocount = 0
		small = 0
		ratio1 = 0
		uvalues1 = 0
		cenlat1 = 0
		cenlon1 = 0
		size1 = 0
		ephi1 = 0
		        
		for isoline in isoline_list:
			#[iso_lon,iso_lat] = mm(isoline[:,0],isoline[:,1],inverse=True)
			[iso_lon,iso_lat] = [isoline[:,0],isoline[:,1]]
			if len(iso_lon)<15:
				print("-----Not analyzing ellipse with",len(iso_lon),"points, continuing...")  # Check for size!
				small = 1
				continue

			# Checking to see if contours are closed (0) or need to be joined (1)  Checking before convert lat/lon to radians
			lon_diff = abs(iso_lon[0] - iso_lon[len(iso_lon)-1])
			lat_diff = abs(iso_lat[0] - iso_lat[len(iso_lat)-1])
			join = 0
			if lon_diff > 1 or lat_diff > 1:
				join = 1
				print("Diffs lat/lon: ",lat_diff,lon_diff)
			iso_lon = np.deg2rad(iso_lon)
			iso_lat = np.deg2rad(iso_lat)

	
			## Now convert it into polar coordinates for ellipse math.
			ex = np.array((np.cos(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))
			ey = np.array((np.sin(iso_lon)*np.cos(iso_lat))/(1+np.sin(iso_lat)))

			## does this contour include the pole?
			overpole = point_inside_polygon(0,0,ex,ey)  #returns true if poly includes the pole, false if not 
			print("It is",overpole, "that the contour includes the pole")
			   
			### contours break if they don't include the pole and need to be reconnected here...
			### first check that contour doesn't include pole, there is more than one contour,... 
			###        ...the contours are not a real split vortex and we didn't get rid of one because it was small...
			if not overpole and len(isoline_list) > 1 and join > 0 and small < 1:
				if isocount > 0:
					ex = np.append(ex,ex2)
					ey = np.append(ey,ey2)
					print("Point didn't include pole - add to these to previous set")
				else:
					ex2 = ex
					ey2 = ey
					print("Point didn't include pole - keeping these to add to next set")
					isocount = 1
					continue

			print("Running ellipse diagnostic now")
			exx,eyy,eaax,ebax,ecenterx,ecentery,ephi = fitEllipseContour(ex,ey)
			
			## Convert back to lat/lon 
			elons = np.where(exx<0,np.where(eyy>0,np.arctan(eyy/exx)+math.pi,np.arctan(eyy/exx)-math.pi),np.arctan(eyy/exx))
			yysinxxlon = eyy/np.sin(elons)
			elats = -2*(np.arctan(yysinxxlon) - (math.pi/4.0))
			elats = np.rad2deg(elats)
			elons = np.rad2deg(elons)
	
			## Still not really sure what this is for...
			for g in range(1,len(elats)):
				if abs(elats[g]-elats[g-1]) > 1.5:
					elats[g] = elats[g-1]
	
			## Center points back to lat/lon
			cenlon = np.where(ecenterx<0,np.where(ecentery>0,np.arctan(ecentery/ecenterx)+math.pi,np.arctan(ecentery/ecenterx)-math.pi),np.arctan(ecentery/ecenterx))
			ysinlon = ecentery/np.sin(cenlon)
			cenlat = np.rad2deg(-2 * (np.arctan(ysinlon) - (math.pi/4.0)))
			cenlon = np.rad2deg(cenlon)
			print("Center of ellipse:",cenlat,"N",cenlon,"E")
	
			## Calculate endpoints of the axes of the vortex, convert to lat/lon
			xa = eaax * np.cos(ephi)
			ya = eaax * np.sin(ephi)
			xb = ebax * np.sin(ephi)
			yb = ebax * np.cos(ephi)
			endx = np.array([ecenterx+xa,ecenterx-xa,ecenterx+xb,ecenterx-xb])
			endy = np.array([ecentery+ya,ecentery-ya,ecentery-yb,ecentery+yb])
			endlon = np.where(endx < 0,np.where(endy>0,np.arctan(endy/endx)+math.pi,np.arctan(endy/endx)-math.pi),np.arctan(endy/endx))
			ysinelon = endy/np.sin(endlon)
			endlat = np.rad2deg(-2 *(np.arctan(ysinelon) - (math.pi/4)))
			endlon = np.rad2deg(endlon)
	
			## Calc great circle distances; (still necessary?) 
			a1gc = great_circle((endlat[0],endlon[0]),(cenlat,cenlon)).km
			a2gc = great_circle((cenlat,cenlon),(cenlat,endlat[1])).km
			b1gc = great_circle((endlat[2],endlon[2]),(cenlat,cenlon)).km
			b2gc = great_circle((cenlat,cenlon),(endlat[3],endlon[3])).km

			ephi = np.rad2deg(ephi)
			## If a1 shorter than b1, adjust phi to correct the orientation angle
			if a1gc < b1gc:
				ephi -= 90
			print("Emetrics phi:",ephi)
			if ephi < -45:
				ephi += 180
			
			ratio = a1gc/b1gc
			if ratio < 1.0:
				ratio = 1.0/ratio
	
			size = math.pi*a1gc*b1gc
			#estr = times[t].strftime("%H %Y%m%d%H")+" "+f'{cenlon:.3f}'+" "+f'{cenlat:.3f}'+" "+f'{a1gc:.3f}'+" "+f'{b1gc:.3f}'+" "+f'{ephi:.3f}'+" "+f'{u.values:.3f}'+" "+f'{temp.values:.3f}'
			#if write_out:
				#fout.write(estr+"\n")

			exy=np.array(list(zip(elons,elats)))
			plt.plot(elons,elats,color='red',transform=ccrs.Geodetic(),linewidth=mem_lw)
			plt.plot(cenlon,cenlat,markersize=4,marker='o',color='red',transform=ccrs.PlateCarree())
			#ax.add_patch(Polygon(exy,closed=True,color='magenta',fill=False,lw=mem_lw,transform=ccrs.PlateCarree()))
			ratio1 = ratio1+ratio
			uvalues1 = uvalues1+u.values
			cenlat1 = cenlat1+cenlat
			cenlon1 = cenlon1+cenlon
			size1 = size1+size
			ephi1 = ephi1+ephi
            
		if len(isoline_list) > 0:
			rat.append(ratio1/len(isoline_list))
			wind.append(uvalues1/len(isoline_list))
			cenl.append(cenlat1/len(isoline_list))
			cenln.append(cenlon1/len(isoline_list))
			sz.append(size1/len(isoline_list))
			ep.append(ephi1/len(isoline_list))
            
		if len(isoline_list) == 0:
			rat.append(np.nan)
			wind.append(np.nan)
			cenl.append(np.nan)
			cenln.append(np.nan)
			sz.append(np.nan)
			ep.append(np.nan)
            
		#print(cenlon)
		#print(cenlat)
		#print(ephi)
		print("right before savefig")
		plot_label = date
        
		if plot_lev == 10:
			plt.savefig(plots_loc10+f"{plot_label}_{plot_lev}hpa.png",format='png')            
#		if plot_lev == 30:
#			plt.savefig(plots_loc30+f"{plot_label}_{plot_lev}hpa.png",format='png')
		if plot_lev == 50:
			plt.savefig(plots_loc50+f"{plot_label}_{plot_lev}hpa.png",format='png')
            
		#plt.savefig(plots_loc+"ellipse"+str(plot_lev)+"_"+str(fhr/6)+".png",format='png')
		print("right after savefig")
		#plt.close()
		plt.clf()
		#quit()
	if plot_lev == 10:
		test_wind = np.empty((366))
		test_wind[:] = np.nan
		for i in range(len(wind)):
			print(i)
			if i in range(83,283):
				continue
			else:
				tes = wind[i][:1]
				lep = tes[0]
				test_wind[i] = lep
                
		ephi_ratio10[0,:] = rat[:]
		ephi_wind10[0,:] = test_wind[:]
		ephi_cenlat10[0,:] = cenl[:]
		ephi_cenlon10[0,:] = cenln[:]
		ephi_size10[0,:] = sz[:]
		ephi10[0,:] = ep[:]

#	if plot_lev == 30:
#		ephi_ratio30[0,124:] = rat[:]
#		ephi_wind30[0,124:] = wind[:]
#		ephi_cenlat30[0,124:] = cenl[:]
#		ephi_size30[0,124:] = sz[:]
#		ephi30[0,124:] = ep[:]

            
	if plot_lev == 50:
        
		test_wind = np.empty((366))
		test_wind[:] = np.nan
		for i in range(len(wind)):
			print(i)
			if i in range(98,289):
				continue
			else:
				tes = wind[i][:1]
				lep = tes[0]
				test_wind[i] = lep
    
		ephi_ratio50[0,:] = rat[:]
		ephi_wind50[0,:] = test_wind[:]
		ephi_cenlat50[0,:] = cenl[:]
		ephi_cenlon50[0,:] = cenln[:]
		ephi_size50[0,:] = sz[:]
		ephi50[0,:] = ep[:]
	#quit()

Opening ERAI data 10 and controur level 30000
Partitioning and averaging data.
First, g data
u data
t data
lat and lon
level: 10
date: 1
height <xarray.DataArray 'g' (lat: 128, lon: 512)>
array([[30822.344, 30821.822, 30821.209, ..., 30823.45 , 30823.162, 30822.805],
       [30823.178, 30822.652, 30822.053, ..., 30824.197, 30823.969, 30823.605],
       [30824.014, 30823.484, 30822.943, ..., 30824.984, 30824.752, 30824.379],
       ...,
       [28575.8  , 28575.555, 28575.316, ..., 28576.6  , 28576.322, 28576.059],
       [28597.746, 28597.58 , 28597.418, ..., 28598.293, 28598.107, 28597.924],
       [28622.266, 28622.19 , 28622.12 , ..., 28622.51 , 28622.426, 28622.344]],
      dtype=float32)
Coordinates:
    time     float64 1.0
    hour     float64 0.0
    lev      int32 10
  * lat      (lat) float32 0.35087654 1.0526296 1.7543826 ... 88.76695 89.46282
  * lon      (lon) float32 0.0 0.70312524 1.4062505 ... 358.59387 359.297
Attributes:
    mapping:          cylindrical_equidistant_p

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.04083048083463581 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.08793473 0.01148008]
old angle of rotation = 3.7193690760785247
old axes = [0.25609887 0.33019779]
center = [0.08793473 0.01148008]
angle of rotation = -86.28063092392148
axes = [0.33019779 0.25609887]
0.3301977935556687 0.2560988710532118 0.08793472795218282 0.011480081041956709 -1.505881090320467
Center of ellipse: 79.86442740335036 N 7.4380285209898025 E
Emetrics phi: -86.28063092392148
right before savefig
right after savefig
date: 2
height <xarray.DataArray 'g' (lat: 128, lon: 512)>
array([[30819.791, 30819.266, 30818.65 , ..., 30820.896, 30820.611, 30820.252],
       [30820.607, 30820.082, 30819.48 , ..., 30821.625, 30821.396, 30821.033],
       [30821.426, 30820.896, 30820.355, ..., 30822.396, 30822.162, 30821.79 ],
       ...,
       [28578.867, 28578.625, 28578.39 , 

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.039557104308528324 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.08829742 0.01090761]
old angle of rotation = 3.449822247173288
old axes = [0.25525154 0.329911  ]
center = [0.08829742 0.01090761]
angle of rotation = -86.55017775282671
axes = [0.329911   0.25525154]
0.3299110018837791 0.2552515364685867 0.08829742348006733 0.010907610078232096 -1.5105855699731732
Center of ellipse: 79.83172226264422 N 7.042218961437035 E
Emetrics phi: -86.55017775282671
right before savefig
right after savefig
date: 3
height <xarray.DataArray 'g' (lat: 128, lon: 512)>
array([[30817.283, 30816.756, 30816.139, ..., 30818.393, 30818.105, 30817.746],
       [30818.082, 30817.555, 30816.953, ..., 30819.1  , 30818.871, 30818.508],
       [30818.883, 30818.354, 30817.814, ..., 30819.855, 30819.62 , 30819.246],
       ...,
       [28582.80

...

range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.052936184238177475 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.05180576 0.04022903]
old angle of rotation = 13.797131576614484
old axes = [0.22098398 0.30200303]
center = [0.05180576 0.04022903]
angle of rotation = -76.20286842338551
axes = [0.30200303 0.22098398]
0.30200303349947943 0.2209839755638755 0.05180576335261729 0.04022903013218765 -1.3299909534520975
Center of ellipse: 82.49454751532507 N 37.83061891813534 E
Emetrics phi: -76.20286842338551
right before savefig
right after savefig
date: 365
height <xarray.DataArray 'g' (lat: 128, lon: 512)>
array([[23659.871, 23660.006, 23660.162, ..., 23659.914, 23659.832, 23659.832],
       [23659.61 , 23659.682, 23659.797, ..., 23659.713, 23659.656, 23659.594],
       [23659.398, 23659.393, 23659.51 , ..., 23659.62 , 23659.504, 23659.377],
       ...,
       [22163

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.05238167603925348 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.05265926 0.03941723]
old angle of rotation = 13.544010771379968
old axes = [0.22101414 0.3027458 ]
center = [0.05265926 0.03941723]
angle of rotation = 13.544010771379966
axes = [0.22101414 0.3027458 ]
0.22101414315207984 0.30274580305504367 0.05265926491201921 0.039417234392431534 0.23638758188615738
Center of ellipse: 82.47326431476773 N 36.81609159072352 E
Emetrics phi: -76.45598922862004
right before savefig
right after savefig
date: 366
height <xarray.DataArray 'g' (lat: 128, lon: 512)>
array([[23659.12 , 23659.248, 23659.402, ..., 23659.174, 23659.088, 23659.084],
       [23658.863, 23658.932, 23659.045, ..., 23658.979, 23658.918, 23658.852],
       [23658.66 , 23658.65 , 23658.766, ..., 23658.89 , 23658.771, 23658.64 ],
       ...,
       [2215

/kt11/ktyle/anaconda3_2001/envs/jun20/lib/python3.7/site-packages/ipykernel_launcher.py:85: DeprecationWarning: The outline_patch property is deprecated. Use GeoAxes.spines['geo'] or the default Axes properties instead.


range(18000, 33500, 250)
in try
Number of ellipses:  1
in elipse ...
Number of ellipses:  1
Diffs lat/lon:  0.05189012809051974 359.2969970703125
It is True that the contour includes the pole
Running ellipse diagnostic now
old center = [0.05336717 0.03876208]
old angle of rotation = 13.250368174830058
old axes = [0.22101788 0.30328372]
center = [0.05336717 0.03876208]
angle of rotation = 13.250368174830058
axes = [0.22101788 0.30328372]
0.22101787544021184 0.30328372097343775 0.053367173850843774 0.03876208419690612 0.23126255175225616
Center of ellipse: 82.45261842037313 N 35.99196237225418 E
Emetrics phi: -76.74963182516994
right before savefig
right after savefig


In [8]:
ephi10

array([[ 93.71936908,  93.44982225,  93.20763507,  93.0684302 ,
         92.83138283,  92.67553684,  92.49833337,  92.35809701,
         92.23940101,  92.14980771,  92.09298161,  92.0342181 ,
         91.99611242,  91.94077693,  91.94943895,  91.97789737,
         92.03562718,  92.15079451,  92.26742569,  92.37557625,
         92.50932462,  92.59716584,  92.76380727,  92.9858586 ,
         93.17929851,  93.39206766,  93.60470016,  93.85992318,
         94.12419591,  94.38418288,  94.64194516,  94.90439847,
         95.19501303,  95.47972746,  95.7397463 ,  96.10599674,
         96.41348317,  96.69536192,  97.01476172,  97.30491172,
         97.65796191,  97.97566084,  98.26383646,  98.56123069,
         98.84337872,  99.20252087,  99.48128109,  99.74588028,
        100.05556075, 100.31691667, 100.54642981, 100.82029225,
        101.03282287, 101.23627826, 101.46330683, 101.63789915,
        101.72914627, 101.93316356, 102.02431722, 102.15017418,
        102.21638579, 102.30209158, 102.

In [9]:
test_wind = np.empty((366))
test_wind[:] = np.nan
for i in range(len(wind)):
    print(i)
    if i in range(83,283):
        continue
    else:
        tes = wind[i][:1]
        lep = tes[0]
        test_wind[i] = lep

print(test_wind)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

TypeError: 'float' object is not subscriptable

In [12]:
pickle.dump(ephi_ratio50, open("Climo_ratio50.p", 'wb'))
pickle.dump(ephi_wind50, open("Climo_wind50.p", 'wb')) 
pickle.dump(ephi_cenlat50, open("Climo_cenlat50.p", 'wb'))
pickle.dump(ephi_size50, open("Climo_size50.p", 'wb'))
pickle.dump(ephi50, open("Climo_ephi50.p", 'wb'))

pickle.dump(ephi_ratio10, open("Climo_ratio10.p", 'wb'))
pickle.dump(ephi_wind10, open("Climo_wind10.p", 'wb')) 
pickle.dump(ephi_cenlat10, open("Climo_cenlat10.p", 'wb'))
pickle.dump(ephi_size10, open("Climo_size10.p", 'wb'))
pickle.dump(ephi10, open("Climo_ephi10.p", 'wb'))


nan

In [27]:
#get mean values of these

rat10 = np.nanmean(ephi_ratio10)
print("Ratio 10 = ", rat10)
print("Max = ", np.nanmax(ephi_ratio10))
print("Min = ", np.nanmin(ephi_ratio10))
print("#####################")

wind10 = np.nanmean(ephi_wind10)
print("Wind 10 = ", wind10)
print("Max = ", np.nanmax(ephi_wind10))
print("Min = ", np.nanmin(ephi_wind10))
print("#####################")

cenl10 = np.nanmean(ephi_cenlat10)
print("Center Latitude 10 = ", cenl10)
print("Max = ", np.nanmax(ephi_cenlat10))
print("Min = ", np.nanmin(ephi_cenlat10))
print("#####################")

cenln10 = np.nanmean(ephi_cenlon10)
print("Center Longitude 10 = ", cenln10)
print("Max = ", np.nanmax(ephi_cenlon10))
print("Min = ", np.nanmin(ephi_cenlon10))
print("#####################")

size10 = np.nanmean(ephi_size10)
print("Max = ", np.nanmax(ephi_size10))
print("Min = ", np.nanmin(ephi_size10))
print("Size 10 = ", size10)


Ratio 10 =  1.2308585854192278
Max =  1.3899944188617004
Min =  0.5891840140146883
#####################
Wind 10 =  25.557737014379846
Max =  36.76875305175781
Min =  4.965406894683838
#####################
Center Latitude 10 =  77.70713801435988
Max =  85.69986143992428
Min =  38.952638458039985
#####################
Center Longitude 10 =  24.63979222308453
Max =  43.860736065539825
Min =  5.146474739200529
#####################
Max =  42213253.667494595
Min =  329078.01752466185
Size 10 =  27639610.812978026


In [ ]:
print("#####################")
rat50 = np.nanmean(ephi_ratio50)
print("Ratio 50 = ", rat50)
wind50 = np.nanmean(ephi_wind50)
print("Wind 50 = ", wind50)
cenl50 = np.nanmean(ephi_cenlat50)
print("Center Latitude 50 = ", cenl50)
cenln50 = np.nanmean(ephi_cenlon50)
print("Center Longitude 50 = ", cenln50)
size50 = np.nanmean(ephi_size50)
print("Size 50 = ", size50)